In [40]:
# Business Data Transformation & Analytics using Pandas and NumPy

# This notebook presents an end-to-end data transformation and analytics solution using Python, Pandas, and NumPy.
# The goal is to ingest raw JSON transaction data generated from an online retail platform, transform it into a clean and structured analytical dataset, and answer key business questions required by leadership and reporting teams.


# Objective

# The objective of this analysis is to:
# - Ingest raw JSON transaction data
# - Transform it into a clean, flat, and structured dataset
# - Ensure one row represents one product sold
# - Apply revenue and business logic using Pandas and NumPy
# - Generate analytics-ready data for dashboards and KPIs

In [41]:
# Phase 1: Data Ingestion
# - Load JSON data
# - Inspect schema and structure
# - Identify nested fields

In [42]:
import pandas as pd 
import numpy as np
import json

with open("demo_json.json", "r", encoding="utf-8") as file:
    raw_data = json.load(file)

# Inspect structure of a single order record
raw_data[0]

{'order_id': 'ORD1001',
 'order_date': '2025-01-10',
 'customer': {'customer_id': 'CUST01',
  'name': 'Rahul Sharma',
  'city': 'Delhi'},
 'items': [{'product_id': 'P101',
   'product_name': 'Laptop',
   'category': 'Electronics',
   'price': 55000,
   'quantity': 1},
  {'product_id': 'P102',
   'product_name': 'Mouse',
   'category': 'Electronics',
   'price': 500,
   'quantity': 2}],
 'payment': {'method': 'Credit Card', 'status': 'Success'}}

In [43]:
# Phase 2: DataFrame Creation
# - Convert JSON into DataFrames
# - Flatten nested fields
# - Expand arrays so that one row represents one product per order

In [44]:
# Create base DataFrame from raw JSON

order_df = pd.json_normalize(raw_data)

order_df

,order_id,order_date,items,customer.customer_id,customer.name,customer.city,payment.method,payment.status
0,ORD1001,2025-01-10,"[{'product_id': 'P101', 'product_name': 'Lapto...",CUST01,Rahul Sharma,Delhi,Credit Card,Success
1,ORD1002,2025-01-11,"[{'product_id': 'P103', 'product_name': 'Mobil...",CUST02,Sneha Verma,Mumbai,UPI,Success
2,ORD1003,2025-01-12,"[{'product_id': 'P201', 'product_name': 'Dinin...",CUST03,Amit Patel,Pune,Cash,Failed
3,ORD1004,2025-01-13,"[{'product_id': 'P104', 'product_name': 'Headp...",CUST04,Neha Singh,Bangalore,Debit Card,Success
4,ORD1005,2025-01-14,"[{'product_id': 'P301', 'product_name': 'Offic...",CUST05,Karan Mehta,Delhi,Net Banking,Success
5,ORD1006,2025-01-15,"[{'product_id': 'P105', 'product_name': 'Smart...",CUST06,Pooja Iyer,Chennai,UPI,Success


In [45]:
# Seprate items so that each product becomes a separate row

product_sepratation = order_df.explode("items")

product_sepratation

,order_id,order_date,items,customer.customer_id,customer.name,customer.city,payment.method,payment.status
0,ORD1001,2025-01-10,"{'product_id': 'P101', 'product_name': 'Laptop...",CUST01,Rahul Sharma,Delhi,Credit Card,Success
0,ORD1001,2025-01-10,"{'product_id': 'P102', 'product_name': 'Mouse'...",CUST01,Rahul Sharma,Delhi,Credit Card,Success
1,ORD1002,2025-01-11,"{'product_id': 'P103', 'product_name': 'Mobile...",CUST02,Sneha Verma,Mumbai,UPI,Success
2,ORD1003,2025-01-12,"{'product_id': 'P201', 'product_name': 'Dining...",CUST03,Amit Patel,Pune,Cash,Failed
2,ORD1003,2025-01-12,"{'product_id': 'P202', 'product_name': 'Chair'...",CUST03,Amit Patel,Pune,Cash,Failed
3,ORD1004,2025-01-13,"{'product_id': 'P104', 'product_name': 'Headph...",CUST04,Neha Singh,Bangalore,Debit Card,Success
4,ORD1005,2025-01-14,"{'product_id': 'P301', 'product_name': 'Office...",CUST05,Karan Mehta,Delhi,Net Banking,Success
5,ORD1006,2025-01-15,"{'product_id': 'P105', 'product_name': 'Smart ...",CUST06,Pooja Iyer,Chennai,UPI,Success


In [46]:
# Normalize product details from items column

items_df = pd.json_normalize(product_sepratation["items"])

items_df

,product_id,product_name,category,price,quantity
0,P101,Laptop,Electronics,55000,1
1,P102,Mouse,Electronics,500,2
2,P103,Mobile Phone,Electronics,32000,1
3,P201,Dining Table,Furniture,18000,1
4,P202,Chair,Furniture,3500,4
5,P104,Headphones,Electronics,2500,2
6,P301,Office Chair,Furniture,12000,1
7,P105,Smart Watch,Electronics,9000,2


In [47]:
# Drop original items column and concatenate product details

final_df = pd.concat([product_sepratation.drop(columns=["items"]).reset_index(drop=True), 
                      items_df.reset_index(drop=True)],
                     axis=1)

final_df

,order_id,order_date,customer.customer_id,customer.name,customer.city,payment.method,payment.status,product_id,product_name,category,price,quantity
0,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,P101,Laptop,Electronics,55000,1
1,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,P102,Mouse,Electronics,500,2
2,ORD1002,2025-01-11,CUST02,Sneha Verma,Mumbai,UPI,Success,P103,Mobile Phone,Electronics,32000,1
3,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,P201,Dining Table,Furniture,18000,1
4,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,P202,Chair,Furniture,3500,4
5,ORD1004,2025-01-13,CUST04,Neha Singh,Bangalore,Debit Card,Success,P104,Headphones,Electronics,2500,2
6,ORD1005,2025-01-14,CUST05,Karan Mehta,Delhi,Net Banking,Success,P301,Office Chair,Furniture,12000,1
7,ORD1006,2025-01-15,CUST06,Pooja Iyer,Chennai,UPI,Success,P105,Smart Watch,Electronics,9000,2


In [48]:
# Phase 3: Data Cleaning
# - Handle missing or invalid values
# - Validate data types
# - Standardize column names

In [49]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   order_id              8 non-null      object
 1   order_date            8 non-null      object
 2   customer.customer_id  8 non-null      object
 3   customer.name         8 non-null      object
 4   customer.city         8 non-null      object
 5   payment.method        8 non-null      object
 6   payment.status        8 non-null      object
 7   product_id            8 non-null      object
 8   product_name          8 non-null      object
 9   category              8 non-null      object
 10  price                 8 non-null      int64 
 11  quantity              8 non-null      int64 
dtypes: int64(2), object(10)
memory usage: 900.0+ bytes


In [50]:
final_df.columns = (
    final_df.columns
    .str.lower()
    .str.replace(".", "_")
    .str.replace("customer_customer_", "customer_", regex=False)
)

final_df = final_df.rename(columns={
    "category": "product_category",
    "quantity": "quantity_sold"
})

final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   order_id          8 non-null      object
 1   order_date        8 non-null      object
 2   customer_id       8 non-null      object
 3   customer_name     8 non-null      object
 4   customer_city     8 non-null      object
 5   payment_method    8 non-null      object
 6   payment_status    8 non-null      object
 7   product_id        8 non-null      object
 8   product_name      8 non-null      object
 9   product_category  8 non-null      object
 10  price             8 non-null      int64 
 11  quantity_sold     8 non-null      int64 
dtypes: int64(2), object(10)
memory usage: 900.0+ bytes


In [51]:
price_array = final_df["price"].to_numpy()
quantity_array = final_df["quantity_sold"].to_numpy()

# Finding Total Amount by using numpy
total_amount_array = np.multiply(price_array, quantity_array)

final_df["total_amount"] = total_amount_array

final_df[["product_name", "price", "quantity_sold", "total_amount"]]

,product_name,price,quantity_sold,total_amount
0,Laptop,55000,1,55000
1,Mouse,500,2,1000
2,Mobile Phone,32000,1,32000
3,Dining Table,18000,1,18000
4,Chair,3500,4,14000
5,Headphones,2500,2,5000
6,Office Chair,12000,1,12000
7,Smart Watch,9000,2,18000


In [52]:
# Phase 4: NumPy-based Transformations
# - Perform revenue calculations using NumPy arrays
# - Apply conditional logic using vectorized operations
# - Boolean masking and performance-efficient processing

In [53]:
total_amount_array = final_df["total_amount"].to_numpy()

# Creating High Value Flag for Order which have Total Amount above 20000
high_value_flag = np.where(total_amount_array>20000, "YES", "NO")

final_df["high_value_order_flag"] = high_value_flag

final_df

,order_id,order_date,customer_id,customer_name,customer_city,payment_method,payment_status,product_id,product_name,product_category,price,quantity_sold,total_amount,high_value_order_flag
0,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,P101,Laptop,Electronics,55000,1,55000,YES
1,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Credit Card,Success,P102,Mouse,Electronics,500,2,1000,NO
2,ORD1002,2025-01-11,CUST02,Sneha Verma,Mumbai,UPI,Success,P103,Mobile Phone,Electronics,32000,1,32000,YES
3,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,P201,Dining Table,Furniture,18000,1,18000,NO
4,ORD1003,2025-01-12,CUST03,Amit Patel,Pune,Cash,Failed,P202,Chair,Furniture,3500,4,14000,NO
5,ORD1004,2025-01-13,CUST04,Neha Singh,Bangalore,Debit Card,Success,P104,Headphones,Electronics,2500,2,5000,NO
6,ORD1005,2025-01-14,CUST05,Karan Mehta,Delhi,Net Banking,Success,P301,Office Chair,Furniture,12000,1,12000,NO
7,ORD1006,2025-01-15,CUST06,Pooja Iyer,Chennai,UPI,Success,P105,Smart Watch,Electronics,9000,2,18000,NO


In [54]:
payment_status_array = final_df["payment_status"].to_numpy()

# Creating Order Overall Status based on Payment is Success/Failed
order_status_array = np.where(payment_status_array == "Success", "Completed", "Cancelled")

final_df["order_status"] = order_status_array

final_df.columns

Index(['order_id', 'order_date', 'customer_id', 'customer_name',
       'customer_city', 'payment_method', 'payment_status', 'product_id',
       'product_name', 'product_category', 'price', 'quantity_sold',
       'total_amount', 'high_value_order_flag', 'order_status'],
      dtype='object')

In [55]:
# Phase 5: Business Logic Application
# - Exclude failed payments
# - Calculate total sales amount
# - Identify high-value orders
# - Assign order status flags

In [56]:
business_df = final_df[final_df["payment_status"] == "Success"].copy()

FINAL_COLUMNS = [
    "order_id",
    "order_date",
    "customer_id",
    "customer_name",
    "customer_city",
    "product_name",
    "product_category",
    "quantity_sold",
    "total_amount",
    "payment_method",
    "order_status",
    "high_value_order_flag"
]

business_df = business_df[FINAL_COLUMNS]

business_df.reset_index(drop=True,inplace=True)

business_df

,order_id,order_date,customer_id,customer_name,customer_city,product_name,product_category,quantity_sold,total_amount,payment_method,order_status,high_value_order_flag
0,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Laptop,Electronics,1,55000,Credit Card,Completed,YES
1,ORD1001,2025-01-10,CUST01,Rahul Sharma,Delhi,Mouse,Electronics,2,1000,Credit Card,Completed,NO
2,ORD1002,2025-01-11,CUST02,Sneha Verma,Mumbai,Mobile Phone,Electronics,1,32000,UPI,Completed,YES
3,ORD1004,2025-01-13,CUST04,Neha Singh,Bangalore,Headphones,Electronics,2,5000,Debit Card,Completed,NO
4,ORD1005,2025-01-14,CUST05,Karan Mehta,Delhi,Office Chair,Furniture,1,12000,Net Banking,Completed,NO
5,ORD1006,2025-01-15,CUST06,Pooja Iyer,Chennai,Smart Watch,Electronics,2,18000,UPI,Completed,NO


In [57]:
# Phase 6: Business-Ready Dataset
# - Produce flat, clean, and structured dataset ready for analytics

# Business Questions to Answer:
# 1. Total revenue per city
# 2. Product category with highest revenue
# 3. Count of high-value orders
# 4. Average order value
# 5. Most preferred payment method
# 6. Top customers by revenue
# 7. Completed vs cancelled orders

In [58]:
# 1. Total revenue per city

revenue_per_city = (business_df.groupby("customer_city")["total_amount"].sum().sort_values(ascending=False))

revenue_per_city

customer_city
Delhi        68000
Mumbai       32000
Chennai      18000
Bangalore     5000
Name: total_amount, dtype: int64

In [59]:
# 2. Product category with highest revenue

category_revenue_series = (business_df.groupby("product_category")["total_amount"].sum())

highest_revenue_categories = category_revenue_series[category_revenue_series == category_revenue_series.max()]

highest_revenue_categories

product_category
Electronics    111000
Name: total_amount, dtype: int64

In [60]:
# 3. Count of high-value orders

high_value_order_count = business_df["high_value_order_flag"].map({"YES": 1, "NO": 0}).sum()

high_value_order_count.item()

2

In [61]:
# 4. Average order value

average_order_value = (
    business_df
    .groupby("order_id")["total_amount"]
    .sum()
    .mean()
)

average_order_value.item()

24600.0

In [62]:
# 5. Most preferred payment method

most_preferred_payment_method = business_df["payment_method"].value_counts()[lambda x: x == x.max()]

most_preferred_payment_method

payment_method
Credit Card    2
UPI            2
Name: count, dtype: int64

In [63]:
# 6. Top customers by revenue

top_customers = (business_df.groupby(["customer_id","customer_name"])["total_amount"].sum().sort_values(ascending=False).head(5))

top_customers

customer_id  customer_name
CUST01       Rahul Sharma     56000
CUST02       Sneha Verma      32000
CUST06       Pooja Iyer       18000
CUST05       Karan Mehta      12000
CUST04       Neha Singh        5000
Name: total_amount, dtype: int64

In [64]:
# 7. Completed vs cancelled orders

order_level_status = (
    final_df
    .groupby("order_id")["order_status"]
    .first()
)

order_status_distribution = order_level_status.value_counts()

order_status_distribution

order_status
Completed    5
Cancelled    1
Name: count, dtype: int64